#Récupération des stations hydrologiques et météorologiques importantes

Dans ce fichier, nous récupérons la liste des stations hydrologiques et météorologiques les plus proches des sources les plus en amont des affluents de niveau 1 et 2 de la Garonne

- watercourse_data.csv : coordonnées initiales et finales des cours de France, avec une hiérarchie par rapport aux 6 fleuves principaux de France
- stations.csv         : liste des stations hydrologies, comme obtenu sur Hub'Eau sur toute la France
- station-meteo.json   : liste des stations météo, scrappé depuis l'API MeteoFrance sur toute la France


In [ ]:
import json
import pandas as pd
import geopy.distance

df_stations = pd.read_csv('./data/stations-hydro.csv')
df_watercourse = pd.read_csv('./data/watercourse_data.csv')

with open('./data/station-meteo.json','r') as file :
    df_meteo = pd.DataFrame(json.load(file))
df_meteo = df_meteo[['id_post', 'post_name', 'post_lati','post_long']]

df_sources = df_watercourse[df_watercourse['river_join'] == 'La Garonne'][(df_watercourse['classe'] ==2) | (df_watercourse['classe'] == 1) ][['name','first_point']]
df_stations = df_stations[['code_station','libelle_site','latitude_station','longitude_station']]

In [ ]:
# Liste des stations hydrologiques les plus proches des sources des fleuves
NAME = 1
SOURCE_COORD = 2
stations_affluents = []
for source in df_sources.itertuples() :
    longitude,latitude = tuple(source[2][1:-1].split(','))
    coords_station = [float(latitude),float(longitude)]
    nearest = None
    nearestKm = 30000
    for s in df_stations.itertuples() :
        coords = [s[3],s[4]]
        km = ((coords_station[0]-coords[0])**2 + (coords_station[1]-coords[1])**2)**0.5
        if km <  nearestKm :
            nearestKm = km
            nearest = s
    stations_affluents.append(nearest[1:])
stations_affluents.append(['O972001001','La Garonne à Bordeaux','',''])

In [ ]:
# Liste des stations météo les plus proches des stations hydro identifiées au-dessus
stations_meteo = []
for station in stations_affluents : 
    coords_station = [station[2],station[3]]
    if coords_station == ['',''] :
        continue
    nearest = None
    nearestKm = 30000
    for s in df_meteo.itertuples() :
        coords = [s[3],s[4]]
        km = ((coords_station[0]-coords[0])**2 + (coords_station[1]-coords[1])**2)**0.5
        if km <  nearestKm :
            nearestKm = km
            nearest = s
    stations_meteo.append(nearest[1:])

In [ ]:
df_final = pd.DataFrame(stations_meteo,columns=['code_station','libelle_site','latitude_station','longitude_station'])
df_final.to_csv('liste_stations_affluents_bordeaux_meteo.csv')
df_final = pd.DataFrame(stations_affluents,columns=['code_station','libelle_site','latitude_station','longitude_station'])
df_final.to_csv('liste_stations_affluents_bordeaux_hydro.csv')